# Become A Wise Investor on Lending Club

LendingClub is a US peer-to-peer lending company, headquartered in San Francisco, California. It was the first peer-to-peer lender to register its offerings as securities with the Securities and Exchange Commission (SEC), and to offer loan trading on a secondary market. LendingClub is the world's largest peer-to-peer lending platform.

Given historical data on loans given out with information on whether or not the borrower defaulted (charge-off), I will build a model that can predict wether or nor a borrower will pay back their loan. This way in the future when there is a new potential customer I can assess whether or not they are likely to pay back the loan. The datset can be obtained from [Kaggle](https://www.kaggle.com/wordsforthewise/lending-club)


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [2]:
df=pd.read_csv('accepted_2007_to_2018Q4.csv')

C:\Users\Duc Cao\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Data Cleaning

In [3]:
# Remove redundant and leak information
df.drop(["desc","url","id","member_id","funded_amnt","funded_amnt_inv",
         "grade","sub_grade","emp_title","issue_d", "zip_code", "out_prncp", 
         "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp",
         "total_rec_int", "total_rec_late_fee", "recoveries", 
         "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"],
          inplace=True,axis=1)
# Remove columns with more than 50% missing values
df.dropna(axis=1,thresh=df.shape[0]*0.5,inplace=True)

# Select features and labels
features_labels=['loan_amnt', 'term', 'int_rate', 'installment', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status','loan_status',
       'pymnt_plan', 'purpose', 'title', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'initial_list_status',
       'last_credit_pull_d', 'collections_12_mths_ex_med', 'policy_code',
       'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths',
       'delinq_amnt', 'pub_rec_bankruptcies', 'tax_liens']

# The column that directly describes if a loan was paid off on time
labels='loan_status'

In [4]:
df=df[features_labels]

# I can treat the problem as a binary classification one: 'Fully Paid' or 'Charged Off'
df = df[(df['loan_status'] == "Fully Paid") | (df['loan_status'] == "Charged Off")]
status_replace = {"loan_status" : {"Fully Paid": 1,"Charged Off": 0,}}
df = df.replace(status_replace)

In [5]:
df.shape

(1345310, 32)

In [6]:
# Drop columns that contain one true unique value
orig_columns = df.columns
drop_columns = []
for col in orig_columns:
    col_series = df[col].dropna().unique()
    if len(col_series) == 1:
        drop_columns.append(col)
df.drop(drop_columns, axis=1,inplace=True)

# Drop columns that offer very little variablity 
df.drop(['delinq_amnt','acc_now_delinq','collections_12_mths_ex_med',
         'chargeoff_within_12_mths','tax_liens','application_type'],axis=1,inplace=True)


In [7]:
# Remove rows containing missing values 
df.dropna(axis=0,inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1250804 entries, 0 to 2260697
Data columns (total 24 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   loan_amnt             1250804 non-null  float64
 1   term                  1250804 non-null  object 
 2   int_rate              1250804 non-null  float64
 3   installment           1250804 non-null  float64
 4   emp_length            1250804 non-null  object 
 5   home_ownership        1250804 non-null  object 
 6   annual_inc            1250804 non-null  float64
 7   verification_status   1250804 non-null  object 
 8   loan_status           1250804 non-null  int64  
 9   purpose               1250804 non-null  object 
 10  title                 1250804 non-null  object 
 11  addr_state            1250804 non-null  object 
 12  dti                   1250804 non-null  float64
 13  delinq_2yrs           1250804 non-null  float64
 14  earliest_cr_line      1250804 non-

In [9]:
# Exlore object column that contain text
object_columns_df = df.select_dtypes(include=["object"])
print(object_columns_df.head())

# Drop categorical columns that contain too many values

df.drop('addr_state',axis=1,inplace=True)

# Title and purpose columns are repeated information. Remove title column
print(df["title"].value_counts())
print(df["purpose"].value_counts())
df.drop('title',axis=1,inplace=True)

# Extract the year from time stamp features
df['earliest_cr_year'] = df['earliest_cr_line'].apply(lambda date:int(date[-4:]))
df = df.drop('earliest_cr_line',axis=1)
df['last_credit_pull_year'] = df['last_credit_pull_d'].apply(lambda date:int(date[-4:]))
df = df.drop('last_credit_pull_d',axis=1)

# Convert the term feature into numeric data type
df['term'] = df['term'].apply(lambda term: int(term[:3]))

# Convert emp_length column to numeric type data
df['emp_length'] = df['emp_length'].str.replace(r'\+*\syears*','').str.replace('< 1','0').astype(int)

         term emp_length home_ownership verification_status  \
0   36 months  10+ years       MORTGAGE        Not Verified   
1   36 months  10+ years       MORTGAGE        Not Verified   
4   60 months    3 years       MORTGAGE     Source Verified   
5   36 months    4 years           RENT     Source Verified   
6   36 months  10+ years       MORTGAGE        Not Verified   

              purpose               title addr_state earliest_cr_line  \
0  debt_consolidation  Debt consolidation         PA         Aug-2003   
1      small_business            Business         SD         Dec-1999   
4      major_purchase      Major purchase         PA         Jun-1998   
5  debt_consolidation  Debt consolidation         GA         Oct-1987   
6  debt_consolidation  Debt consolidation         MN         Jun-1990   

  initial_list_status last_credit_pull_d  
0                   w           Mar-2019  
1                   w           Mar-2019  
4                   w           Mar-2018  
5         

In [10]:
# Categorical and dummy variables
# Convert home_ownership to dummy variables, but replace NONE and ANY with OTHER
df['home_ownership']=df['home_ownership'].replace(['NONE', 'ANY'], 'OTHER')
dummies = pd.get_dummies(df['home_ownership'],drop_first=True)
df = df.drop('home_ownership',axis=1)
df = pd.concat([df,dummies],axis=1)

# Convert other categorical variables to dummy variables
dummies = pd.get_dummies(df[['verification_status','initial_list_status','purpose' ]],drop_first=True)
df = df.drop(['verification_status','initial_list_status','purpose'],axis=1)
df = pd.concat([df,dummies],axis=1)

In [11]:
# Grab a sample of the dataset
df = df.sample(frac=0.1,random_state=101)

features = df[df.columns.drop('loan_status')]
target = df['loan_status']
df.shape

(125080, 37)

In [12]:
df.reset_index(drop=True,inplace=True)

In [13]:
features = df[df.columns.drop(['loan_status'])]
target = df['loan_status']

## Logistic Model

In [14]:
lr = LogisticRegression(max_iter=500)

# 10-fold cross validation
predictions = cross_val_predict(lr, features, target, cv=10)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (df["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (df["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (df["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (df["loan_status"] == 0)
tn = len(predictions[tn_filter])
# Rates
tpr = tp  / (tp + fn)
fpr = fp  / (fp + tn)

print(tpr)
print(fpr)

0.9833257240407329
0.9292937597632163


In [15]:
# Account for imbalance in the classes
lr = LogisticRegression(class_weight='balanced',max_iter=500)

# 10-fold cross validation
predictions = cross_val_predict(lr, features, target, cv=10)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (df["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (df["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (df["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (df["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp  / (tp + fn)
fpr = fp  / (fp + tn)

print(tpr)
print(fpr)

0.6915060444250353
0.4052865247060758


In [16]:
# To improve FPR, impose a penalty of 10 for misclassifying a 0 and a penalty of 1 for misclassifying a 1
penalty = {0: 10, 1: 1}

lr = LogisticRegression(class_weight=penalty,max_iter=500)
predictions = cross_val_predict(lr, features, target, cv=10)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (df["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (df["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (df["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (df["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.17548682930702503
0.04526021540738305


## Random Forest Algorithm

In [17]:
# Account for the imbalance in the classes
rf = RandomForestClassifier(class_weight="balanced")

# 10-fold cross validation
predictions = cross_val_predict(rf, features, target, cv=10)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (df["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.`
tp_filter = (predictions == 1) & (df["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (df["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (df["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.9810230859320722
0.8816903724410097


## Neural Networks Model

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.constraints import max_norm
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

In [19]:
model = Sequential()

# input layer
model.add(Dense(36,  activation='relu'))
model.add(Dropout(0.2))

# hidden layer
model.add(Dense(18, activation='relu'))
model.add(Dropout(0.2))

# hidden layer
model.add(Dense(9, activation='relu'))
model.add(Dropout(0.2))

# output layer
model.add(Dense(units=1,activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam')

In [20]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=10, shuffle=True,random_state=1)
fp=0
tp=0
fn=0
tn=0

for train_index,test_index in kf.split(features):
    X_train,X_test = features.iloc[train_index].values, features.iloc[test_index].values
    y_train,y_test = target.iloc[train_index].values, target.iloc[test_index].values
    
    # Data Transformation
    scaler = MinMaxScaler()
    X_train=scaler.fit_transform(X_train)
    X_test=scaler.transform(X_test)
    
    
    model.fit(x=X_train, y=y_train, epochs=100,batch_size=250)
    predictions = model.predict_classes(X_test)
    results=confusion_matrix(y_test,predictions)
    
    # False positives.
    fp += results[0][1]

    # True positives.`
    tp += results[1][1]

    # False negatives.
    fn += results[1][0]
    
    # True negatives
    tn += results[0][0]

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)
print(tpr)
print(fpr)

Train on 112572 samples
Epoch 1/100
112572/112572 [==============================] - 2s 20us/sample - loss: 0.4889
Epoch 2/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4608
Epoch 3/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4546
Epoch 4/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4517
Epoch 5/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4500
Epoch 6/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4484
Epoch 7/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4475
Epoch 8/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4457
Epoch 9/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4454
Epoch 10/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4442
Epoch 11/100
112572/112572 [==============================] - 1s 8us/sampl

112572/112572 [==============================] - 1s 7us/sample - loss: 0.4244
Epoch 91/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4240
Epoch 92/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4240
Epoch 93/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4243
Epoch 94/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4239
Epoch 95/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4243
Epoch 96/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4241
Epoch 97/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4238
Epoch 98/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4241
Epoch 99/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4240
Epoch 100/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4244
Train on 11

Epoch 80/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4229
Epoch 81/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4234
Epoch 82/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4226
Epoch 83/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4223
Epoch 84/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4230
Epoch 85/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4229
Epoch 86/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4214
Epoch 87/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4227
Epoch 88/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4233
Epoch 89/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4229
Epoch 90/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4227

112572/112572 [==============================] - 1s 8us/sample - loss: 0.4224
Epoch 70/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4217
Epoch 71/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4222
Epoch 72/100
112572/112572 [==============================] - 1s 9us/sample - loss: 0.4220
Epoch 73/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4214
Epoch 74/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4213
Epoch 75/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4217
Epoch 76/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4215
Epoch 77/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4219
Epoch 78/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4217
Epoch 79/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4220
Epoch 80/100

Epoch 59/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4220
Epoch 60/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4221
Epoch 61/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4224
Epoch 62/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4220
Epoch 63/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4218
Epoch 64/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4223
Epoch 65/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4219
Epoch 66/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4223
Epoch 67/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4225
Epoch 68/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4215
Epoch 69/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4210

...

112572/112572 [==============================] - 1s 7us/sample - loss: 0.4206
Epoch 28/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4209
Epoch 29/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4205
Epoch 30/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4212
Epoch 31/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4208
Epoch 32/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4197
Epoch 33/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4201
Epoch 34/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4200
Epoch 35/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4199
Epoch 36/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4202
Epoch 37/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4202
Epoch 38/100

Epoch 17/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4201
Epoch 18/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4202
Epoch 19/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4199
Epoch 20/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4202
Epoch 21/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4204
Epoch 22/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4191
Epoch 23/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4207
Epoch 24/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4206
Epoch 25/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4193
Epoch 26/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4195
Epoch 27/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4200

112572/112572 [==============================] - 1s 7us/sample - loss: 0.4205
Epoch 7/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4205
Epoch 8/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4213
Epoch 9/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4213
Epoch 10/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4201
Epoch 11/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4203
Epoch 12/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4210
Epoch 13/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4204 0
Epoch 14/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4206
Epoch 15/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4194
Epoch 16/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4209
Epoch 17/100


112572/112572 [==============================] - 1s 8us/sample - loss: 0.4204
Epoch 97/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4202
Epoch 98/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4194
Epoch 99/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4199
Epoch 100/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4205
Train on 112572 samples
Epoch 1/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4199
Epoch 2/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4204
Epoch 3/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4196
Epoch 4/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4199
Epoch 5/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4201
Epoch 6/100
112572/112572 [==============================] - 1s 7us/sample - loss: 

Epoch 86/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4187
Epoch 87/100
112572/112572 [==============================] - 1s 7us/sample - loss: 0.4196
Epoch 88/100
112572/112572 [==============================] - 1s 10us/sample - loss: 0.4192
Epoch 89/100
112572/112572 [==============================] - 1s 9us/sample - loss: 0.4191
Epoch 90/100
112572/112572 [==============================] - 1s 9us/sample - loss: 0.4199
Epoch 91/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4193
Epoch 92/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4189
Epoch 93/100
112572/112572 [==============================] - 1s 9us/sample - loss: 0.4189
Epoch 94/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4182
Epoch 95/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.4192
Epoch 96/100
112572/112572 [==============================] - 1s 8us/sample - loss: 0.419

## Summary

Without accounting for imbalanced classes, the 3 models are good at identifying all the good loans (true positive rate), but also incorrectly identify most of bad loans (false positive rate). Neural networks model proves to be better (lowest FPR).

When I account for imbalanced classes, the logistic model can lower the FPR to 4.7%, and thus lower the risk. Note that this comes at the expense of true positive rate (17%).